# Improved by word2vec 

## 改进思路：1
## 加入停用词，blurb中的简介利用word2vec训练的向量替代，效果比原版提升了10%， 在15代的时候达到最好。loss=, val_loss

## 改进思路：2
## 改变模型架构

In [1]:
import os, sys, re, pickle
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import sklearn
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from gensim.models import Word2Vec
import nltk
from nltk.corpus import stopwords
print(sys.version_info)
for module in tf, mpl, np, pd, sklearn, tf, keras:
    print(module.__name__, module.__version__)

sys.version_info(major=3, minor=7, micro=6, releaselevel='final', serial=0)
tensorflow 2.0.0
matplotlib 3.1.1
numpy 1.16.4
pandas 0.25.3
sklearn 0.22.1
tensorflow 2.0.0
tensorflow_core.keras 2.2.4-tf


In [2]:
ORIGIN_DATA_DIR = os.getcwd()+'/all_fearures/BX-CSV-Dump/'
FILTERED_DATA_DIR = os.getcwd()+'/tmp/'
class DataLoad:
    def __init__(self):
        '''
        books_with_blurbs.csv cloumns: ISBN,text,Author,Year,Publisher,Blurb
        BX-Book-Ratings.csv cloumns: User-ID,ISBN,Book-Rating
        BX-Books.csv cloumns: ISBN,Book-text,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
        BX-Users.csv cloumns: User-ID,Location,Age
        '''
        self.BX_Users = self.load_origin('BX-Users')
        self.BX_Book_Ratings = self.load_origin('BX-Book-Ratings')
        self.Books = self.load_origin('books_with_blurbs', ',')
        #合并三个表
        self.features, self.ISBN2int, self.UserID2int, self.Users, self.blurb2int  = self.get_features()
        self.labels = self.features.pop('Book-Rating')

    def load_origin(self, 
        filename: "根据文件名获取源文件，获取正确得columns、values等值", 
        sep: "因为源文件的分隔方式sep不同，所以通过传参改编分隔方式"="\";\"", 
        )->pd.DataFrame:
        '''
        获取原始数据，第一遍获取后将用pickle保存到本地，方便日后调用
        '''
        try:
            # 从缓存的文件夹FILTERED_DATA_DIR获取基本被过滤后的文件
            pickled_data = pickle.load(open(FILTERED_DATA_DIR+filename+'.p', mode='rb'))
            return pickled_data
        except FileNotFoundError:
            # 如果缓存的文件不存在或者没有，则在源目录ORIGIN_DATA_DIR获取
            all_fearures = pd.read_csv(ORIGIN_DATA_DIR+filename+'.csv', engine='python',sep=sep, encoding='utf-8')
            # \";\"  初始过滤的文件
            # ,      初始不需要过滤的文件
            data_dict = {"\";\"":self.filtrator(all_fearures), ',':all_fearures}
            # 因为没获得处理后的文件，所以我们在获取源文件后可以保存一下处理后的文件
            pickle.dump((data_dict[sep]), open(FILTERED_DATA_DIR+filename+'.p', 'wb'))
            return data_dict[sep]
        except UnicodeDecodeError as e:
            ''' 测试时经常会出现编码错误，如果尝试更换编码方式无效，可以将编码错误的部分位置重新复制粘贴就可以了，这里我们都默认UTF-8'''
            print('UnicodeDecodeError:',e)
        except pd.errors.ParserError as e:
            print("connect error|pandas Error: %s" % e)

    def filtrator(self, 
        f_data: "输入需要进行初步filter的数据"
        )->pd.DataFrame:
        '''
        源文件中的columns和各个值得第一列的第一个字符和最后一列的最后一个字符都带有双引号‘"’,需要将其filter,Location字段当用户Age为null的时候，末尾会有\";NULL字符串 ，直接用切片调整
        '''
        Nonetype_age = 0
        f_data = f_data.rename(columns={f_data.columns[0]:f_data.columns[0][1:], f_data.columns[-1]:f_data.columns[-1][:-1]})
        f_data[f_data.columns[0]] = f_data[f_data.columns[0]].map(lambda v:v[1:] if v!=None else Nonetype_age)
        f_data[f_data.columns[-1]] = f_data[f_data.columns[-1]].map(lambda v:v[:-1] if v!=None else Nonetype_age)
        try:
            f_data = f_data[f_data['Location'].notnull()][f_data[f_data['Location'].notnull()]['Location'].str.contains('\";NULL')]
            f_data['Location'] = f_data['Location'].map(lambda location:location[:-6])
        except:
            pass
        return f_data

    def get_features(self):
        '''
        获取整个数据集的所有features，并对每个文本字段作xxxxx
        User-ID、Location、ISBN、Book-Rating、Title、Author、Year、Publisher、Blurb
        '''
        try:
            # 从缓存的文件夹FILTERED_DATA_DIR获取features的文件
            all_fearures, ISBN2int, UserID2int, Users, blurb2int = pickle.load(open(FILTERED_DATA_DIR+'features.p', mode='rb'))
            return all_fearures, ISBN2int, UserID2int, Users
        except:
            # 将所有的数据组成features大表
            all_fearures = pd.merge(pd.merge(self.BX_Users, self.BX_Book_Ratings), self.Books)
            Users = all_fearures
            # 因为没获得处理后的文件，所以我们在获取源文件后可以保存一下处理后的文件
            # isbn2index userid2index
            all_fearures.pop('Age')
            all_fearures['Title'], title2int = self.feature2int(all_fearures['Title'], 'text')
            all_fearures['Blurb'], blurb2int  = self.feature2int(all_fearures['Blurb'], 'text')
            all_fearures['ISBN'], ISBN2int = self.feature2int(all_fearures['ISBN'], 'word')
            all_fearures['Author'], X2int = self.feature2int(all_fearures['Author'], 'word')
            all_fearures['Publisher'], X2int = self.feature2int(all_fearures['Publisher'], 'word')
            all_fearures['Year'], X2int = self.feature2int(all_fearures['Year'], 'word')
            all_fearures['User-ID'], UserID2int  = self.feature2int(all_fearures['User-ID'], 'word')
            all_fearures['Location'] = self.feature2int(all_fearures['Location'], 'list')
            all_fearures['Book-Rating'] = all_fearures['Book-Rating'].astype('float32')
            pickle.dump((all_fearures, ISBN2int, UserID2int, Users), open(FILTERED_DATA_DIR+'features.p', 'wb'))
            return all_fearures, ISBN2int, UserID2int, Users, blurb2int

    def feature2int(self, 
        feature:'特征值',
        feature_type:'text/word/list'):
        '''
        将文本字段比如title、blurb只取英文单词，并用空格为分隔符，做成一个带index值的集合，并用index值表示各个单词，作为文本得表示
        '''
        pattern = re.compile(r'[^a-zA-Z]')
        filtered_map = {val:re.sub(pattern, ' ', str(val)) for ii,val in enumerate(set(feature)) }
        letter_filter = lambda feature:feature.map({val:re.sub(pattern, ' ', str(val)) for ii,val in enumerate(set(feature)) })
        text_words = set()
        filtered_feature = letter_filter(feature)
        for val in filtered_feature.str.split():
            text_words.update(val)
        text2int = {val:ii for ii, val in enumerate(text_words)}
        text_map = {val:[text2int[row] for row in filtered_map[val].split()][:200] for ii,val in enumerate(set(feature))}
              
        word_map = {val:ii for ii,val in enumerate(set(feature))}
        try:
            cities = set()
            for val in feature.str.split(','):
                cities.update(val)
            city2int = {val:ii for ii, val in enumerate(cities)}
            list_map = {val:[city2int[row] for row in val.split(',')][:3] for ii,val in enumerate(set(feature))}
        except AttributeError :
            list_map = {}

        feature_dict = {
            'text':(feature.map(text_map), text_map),
            'word':(feature.map(word_map), word_map),
            'list':(feature.map(list_map)),
            }
        return feature_dict[feature_type]

    def __del__(self):
        pass
    

origin_DATA = DataLoad()
origin_DATA.features.head()

,User-ID,Location,ISBN,Title,Author,Year,Publisher,Blurb
0,866,"[6235, 579, 6140]",13770,"[2732, 20340, 20672]",1909,64,2234,"[100498, 62786, 1003, 86311, 97903, 118024, 12..."
1,866,"[6235, 579, 6140]",13446,"[1463, 21098, 16110, 16549, 329, 12926, 10988,...",11314,72,2867,"[56570, 52866, 17228, 67961, 44141, 1003, 3674..."
2,8127,"[819, 1898, 6140]",13446,"[1463, 21098, 16110, 16549, 329, 12926, 10988,...",11314,72,2867,"[56570, 52866, 17228, 67961, 44141, 1003, 3674..."
3,12958,"[554, 2975, 3542]",13446,"[1463, 21098, 16110, 16549, 329, 12926, 10988,...",11314,72,2867,"[56570, 52866, 17228, 67961, 44141, 1003, 3674..."
4,6447,"[1494, 2532, 3542]",13446,"[1463, 21098, 16110, 16549, 329, 12926, 10988,...",11314,72,2867,"[56570, 52866, 17228, 67961, 44141, 1003, 3674..."


In [3]:
def prepoces_blurb():
    blurb_word2int = origin_DATA.blurb2int
    blurb_int2word = {tuple(v): i for i,v in blurb_word2int.items()}
    # 在这里我拿到了所有的句子
    blurb_word_list = [blurb_int2word[tuple(val)] for val in origin_DATA.features.Blurb]
    # 过滤停用词
    stop_words = stopwords.words('english')
    for sw in [',', ', ', ' ,', '«', '»', 'même', 'à', 'orée', '…', 'l', '', ' ', '``', '#']:
        stop_words.append(sw)
    new_sentences = []
    for sentences in blurb_word_list:
        sen_list = nltk.word_tokenize(sentences)
        new_wrod_list = []
        for word_ in sen_list:
            if word_ not in stop_words:
                new_wrod_list.append(word_)
        new_sentences.append(new_wrod_list)
    with open("./blurb.txt", "w", encoding='utf-8') as f:
        for sen in new_sentences:
            for word_ in sen:
                f.write(word_)
                f.write(' ')
            f.write('\n')
if not os.path.exists('./blurb.txt'):
    prepoces_blurb()

In [4]:
blurb_new = pd.read_csv('./blurb.txt', names=['Blurb'], sep='\t')
print(blurb_new.shape)
print(blurb_new.head(10))

(172102, 1)
                                               Blurb
0  Here first time paperback outstanding military...
1  The fascinating true story world 's deadliest ...
2  The fascinating true story world 's deadliest ...
3  The fascinating true story world 's deadliest ...
4  The fascinating true story world 's deadliest ...
5  Winnie Helen kept others worst secrets fifty y...
6  Winnie Helen kept others worst secrets fifty y...
7  Winnie Helen kept others worst secrets fifty y...
8  Winnie Helen kept others worst secrets fifty y...
9  Winnie Helen kept others worst secrets fifty y...


In [5]:
blurb_word_list = []
blurb_set = set()
for sen in blurb_new.values:
    sen_list = nltk.word_tokenize(sen[0])
    new_word_list_ = []
    # 对于某些异常的句子，应该将其处理掉
    if len(sen_list) > 600:
        sen_list = sen_list[:30]
    for word_ in sen_list:
        new_word_list_.append(word_)
        blurb_set.add(word_)
    blurb_word_list.append(new_word_list_)

In [6]:
length_blurb = [len(i) for i in blurb_word_list]
print(np.mean(np.array(length_blurb)))

88.63560562921988


In [7]:
# 建立字典
blurb_dict = {v:i for i, v in enumerate(blurb_set)}
print(len(blurb_word_list))

172102


In [8]:
w = []
for i in blurb_word_list:
    for word in i:
        w.append(word)
print(len(set(w)))
print(len(blurb_dict))
print(list(blurb_dict.items())[:100])

159655
159655
[('serotonin', 0), ('T.C', 1), ('rummages', 2), ('Loved', 3), ('BRANDED', 4), ('Corbell', 5), ('masalas', 6), ('enemy-the', 7), ('THUD', 8), ('Columbian', 9), ('Adept', 10), ('heroin-addicted', 11), ('Belfry', 12), ('tercera', 13), ('Brandewyne', 14), ('poachers', 15), ('interessieren', 16), ('McIntyre', 17), ('scandals', 18), ('1590s', 19), ('mid-70s', 20), ('six-shooter', 21), ('double-deals', 22), ('job-hopping', 23), ('Bit', 24), ('hard.,2', 25), ('mastery', 26), ('entstellt', 27), ('McCammon', 28), ('teaser', 29), ('shrewishness', 30), ('Guilty—Until', 31), ('Memo', 32), ('Gates-Margolis', 33), ('Brethren', 34), ('Rapscallions', 35), ('damage', 36), ('espera', 37), ('high.Most', 38), ('uninfected', 39), ('sguardi', 40), ('forests', 41), ('Carnivores', 42), ('ElimiDATE', 43), ('marshmallow', 44), ('Nerone', 45), ('Zadie', 46), ('situada', 47), ('fertility', 48), ('Roberto', 49), ('murmurings—pulsing', 50), ('read-haired', 51), ('contamination', 52), ('Phar', 53), ('Re

In [9]:
def trainWord2Vec():#训练word2vec模型并存储
    train_sentences = blurb_word_list
    # 这里min_count会对字典做截断，如果min_coun=5,那么出现次数小于5的词会被丢弃，需要更新原本的字典
    model=Word2Vec(sentences=train_sentences,size=100,min_count=0,window=5)
    model.save('./blurb_word2vec.model')
trainWord2Vec()

In [10]:
model=Word2Vec.load('./blurb_word2vec.model')
print(model.wv['computer'])

[ 5.9754843e-01  2.8008566e+00 -2.8137808e+00 -6.1239737e-01
  3.9987683e+00 -1.1176540e+00  6.1179692e-01  4.6012250e-01
  3.8250690e+00 -5.7861561e-01 -8.1807993e-02 -1.6565148e+00
 -1.1872522e+00 -1.2961287e+00 -3.4909885e+00  3.9388621e+00
  8.2338262e-01 -5.0523543e-01 -1.8289630e+00  1.2028067e+00
 -2.5206101e-01  1.7424383e+00 -1.7303660e+00 -4.7074932e-01
  1.2209388e+00  1.7551062e+00 -7.8166610e-01  3.3624601e-02
  1.7001582e+00  1.2212100e+00 -3.5618749e-01  8.4452039e-01
  1.6091290e+00  4.4228479e-01 -1.5335855e+00  5.4612851e-01
 -2.3034046e+00  3.8279355e+00  5.9133625e-01 -1.7215645e+00
 -1.2617992e+00 -3.0895033e+00 -1.6020856e+00 -1.2805055e+00
 -7.2305924e-01 -6.5627724e-01 -2.4727347e+00 -9.4625473e-01
 -1.3531775e+00  6.0299454e+00  1.3728358e+00  2.9687726e+00
 -5.1364779e+00  1.6492635e+00  1.9755188e+00  1.0195773e+00
 -6.9438386e-01 -1.7585691e+00  2.0761497e+00 -4.0419024e-01
  1.7259640e+00  7.5109559e-01 -3.3816978e-01 -4.8804202e+00
  3.7444451e-01 -2.43156

In [11]:
blurb_word2vec_martics = np.zeros((len(blurb_dict), 100))
for i, value in blurb_dict.items():
    try:
        blurb_word2vec_martics[value, :] = model.wv[i]
    except:
        print(i ,value)
        print(model.wv[i])
        exit(1)
print(blurb_word2vec_martics.shape)
# 添加pad字符对于的向量
blurb_word2vec_martics = np.insert(blurb_word2vec_martics, blurb_word2vec_martics.shape[0], axis=0, values=np.zeros(shape=(1, 100)))
print(blurb_word2vec_martics.shape)

(159655, 100)
(159656, 100)


In [12]:
# 将blurb_word_list变为数字列表
# 增加一个pad字符
blurb_dict['<padd>'] = len(blurb_dict)
pad_index = blurb_dict['<padd>']
print('pad_index = ', pad_index)
blurb_int_list = []
for one_blurb in blurb_word_list:
    # 补齐和去长
    one_blurb = one_blurb[:200]
    if len(one_blurb)< 200:
        one_blurb = one_blurb + ['<padd>'] * (200- len(one_blurb))
    
    # 转化为数字列表
    temp_list = []
    for word in one_blurb:
        temp_list.append(blurb_dict[word])
    blurb_int_list.append(temp_list)

blurb = np.array(blurb_int_list)
print(blurb.shape)

pad_index =  159655
(172102, 200)


In [13]:
# user-id的字典,总共有28836个用户
all_user = len(set(origin_DATA.features['User-ID']))
new_user_id = {val: i for i, val in enumerate(set(origin_DATA.features['User-ID']))}
print('all user id = ', all_user)
# location的数量=7573(从0开始的)
all_location = max([j for i in origin_DATA.features.Location for j in i]) +1 
print('all location = ', all_location)

all user id =  28836
all location =  7573


In [14]:
# ISBN总数
all_isbn = len(set(origin_DATA.features['ISBN']))
print('all isbn = ', all_isbn)
# author总数
all_author = len(set(origin_DATA.features['Author']))
print('all author = ', all_author)
# year总数
all_year = len(set(origin_DATA.features['Year']))
print('all year = ', all_year)
# publish总数
all_publisher = len(set(origin_DATA.features['Publisher']))
print('all publisher = ', all_publisher)
# title中所有单词总数
all_title_words = max([j for i in origin_DATA.features.Title for j in i]) +1
print('all title words = ', all_title_words)
# blurb中所有单词总数
all_blurb_words = len(blurb_dict)
print('all blurb words = ', all_blurb_words)

all isbn =  38036
all author =  15196
all year =  81
all publisher =  2909
all title words =  23815
all blurb words =  159656


In [15]:
m = len(origin_DATA.features['Location'])
# 对title进行补齐

title = []
for ti in origin_DATA.features['Title'].values:
    if len(ti) > 10:
        ti = ti[:10]
    if len(ti)<10:
        ti = ti + [all_title_words] * (10 - len(ti))

    title.append(ti)
all_title_words += 1
title = np.array(title)
print(title.shape)
print(title[:10])

(172102, 10)
[[ 2732 20340 20672 23815 23815 23815 23815 23815 23815 23815]
 [ 1463 21098 16110 16549   329 12926 10988   623 16549  4583]
 [ 1463 21098 16110 16549   329 12926 10988   623 16549  4583]
 [ 1463 21098 16110 16549   329 12926 10988   623 16549  4583]
 [ 1463 21098 16110 16549   329 12926 10988   623 16549  4583]
 [21098  7472 10948 23415  9289 23815 23815 23815 23815 23815]
 [21098  7472 10948 23415  9289 23815 23815 23815 23815 23815]
 [21098  7472 10948 23415  9289 23815 23815 23815 23815 23815]
 [21098  7472 10948 23415  9289 23815 23815 23815 23815 23815]
 [21098  7472 10948 23415  9289 23815 23815 23815 23815 23815]]


In [16]:
import math
def get_train_val_test():  
    # 对location取3位数
    loca = np.zeros((m, 3))
    for i in range(m):
        loca[i] = np.array(origin_DATA.features['Location'][i])
    print(loca[:-2])

    input_features = [origin_DATA.features['User-ID'].to_numpy(), loca, 
                      origin_DATA.features['ISBN'].to_numpy(), origin_DATA.features['Author'].to_numpy(),
                     origin_DATA.features['Year'].to_numpy(), origin_DATA.features['Publisher'].to_numpy(), 
                     title, blurb]
    labels = origin_DATA.labels.to_numpy()
    # 分割数据集以及shuffle
    np.random.seed(100)
    number_features = len(input_features)
    shuffle_index = np.random.permutation(m)
    shuffle_train_index = shuffle_index[:math.ceil(m*0.96)]
    shuffle_val_index = shuffle_index[math.ceil(m*0.96): math.ceil(m*0.98)]
    shuffle_test_index = shuffle_index[math.ceil(m*0.98):]
    train_features = [input_features[i][shuffle_train_index] for i in range(number_features)]
    train_labels = labels[shuffle_train_index]
    val_features = [input_features[i][shuffle_val_index] for i in range(number_features)]
    val_lables = labels[shuffle_val_index]
    test_features = [input_features[i][shuffle_test_index] for i in range(number_features)]
    test_lables = labels[shuffle_test_index]
    return train_features, train_labels, val_features, val_lables, test_features, test_lables

In [17]:

train_features, train_labels, val_features, val_lables, test_features, test_lables = get_train_val_test()
print(train_features[0].shape)
print(val_features[0].shape)
print(test_features[0].shape)

[[6235.  579. 6140.]
 [6235.  579. 6140.]
 [ 819. 1898. 6140.]
 ...
 [4974.  973. 3542.]
 [6524.  705. 3542.]
 [6524.  705. 3542.]]
(165218,)
(3442,)
(3442,)


In [18]:
def get_inputs():
    # 用户特征输入
    user_id = keras.layers.Input(shape=(1,), dtype='int32', name='user_id_input')
    user_location = keras.layers.Input(shape=(3,), dtype='int32', name='user_location_input')
    
    # 书籍特征输入
    book_isbn = keras.layers.Input(shape=(1,),  dtype='int32', name='book_isbn_input')
    book_author = keras.layers.Input(shape=(1,),  dtype='int32', name='book_author_input')
    book_year = keras.layers.Input(shape=(1,),  dtype='int32', name='book_year_input')
    book_publisher = keras.layers.Input(shape=(1,),  dtype='int32', name='book_publisher_input')  
    book_title = keras.layers.Input(shape=(10, ), dtype='int32', name='book_title_input')
    book_blurb = keras.layers.Input(shape=(200, ), dtype='int32', name='book_blurb_input')
    return user_id, user_location, book_isbn, book_author, book_year, book_publisher, book_title, book_blurb

In [19]:
# 嵌入矩阵的维度
embed_dim = 4
embed_dim_title = 16
# 这里的100跟上文的预训练模型size应一致
embed_dim_blurb = 100

In [20]:
def user_embed_layer(u_id, u_loca):
    user_id_embedd = keras.layers.Embedding(all_user, embed_dim, name='user_id_embedding')(u_id)
    user_loca_embedd = keras.layers.Embedding(all_location, embed_dim , name='user_loca_embedding')(u_loca)
    return user_id_embedd, user_loca_embedd

In [27]:
def book_emded_layer(b_isbn, b_atuhor, b_year, b_publisher, b_title, b_blurb):
    book_isbn_embedd = keras.layers.Embedding(all_isbn, embed_dim, name='book_isbn_embedding')(b_isbn)
    book_author_embedd = keras.layers.Embedding(all_author, embed_dim, name='book_author_embedding')(b_atuhor)
    book_year_embedd = keras.layers.Embedding(all_year, embed_dim, name='book_year_embedding')(b_year)
    book_publisher_embedd = keras.layers.Embedding(all_publisher, embed_dim, name='book_publisher_embedding')(b_publisher)
    
    book_title_embedd = keras.layers.Embedding(all_title_words, embed_dim_title, name='book_title_embedding')(b_title)
    # 加载预训练模型
    book_blurb_embedd = keras.layers.Embedding(all_blurb_words, embed_dim_blurb, name='book_blurb_embedding', weights=[blurb_word2vec_martics], trainable=True)(b_blurb)
    return book_isbn_embedd, book_author_embedd, book_year_embedd, book_publisher_embedd, book_title_embedd, book_blurb_embedd

In [28]:
def get_user_feature(u_id_embedd, u_loca_embedd):
    u_id_layer = keras.layers.Dense(32, activation='relu', kernel_regularizer=tf.nn.l2_loss, name='u_id_dense')(u_id_embedd)
#     u_id_layer_drop = keras.layers.Dropout(rate=0.5, name='u_id_layer_drop')(u_id_layer)
    # u_id_layer.shape = (?, 1, 32)
    # u_loca_layer.shape = (?, 32)
    # 这里可以再加个Dense
    u_loca_layer = keras.layers.Bidirectional(tf.keras.layers.LSTM(16, name='u_loca_lstm'), merge_mode='concat')(u_loca_embedd)
    u_loca_lstm_dense = keras.layers.Dense(32, activation='relu', kernel_regularizer=tf.nn.l2_loss, name='u_loca_lstm_dense')(u_loca_layer)
    u_id_reshape = keras.layers.Reshape([32])(u_id_layer)
    u_combine = keras.layers.concatenate([u_id_reshape, u_loca_lstm_dense],axis=1, name='u_combine')
    print(u_combine.shape)
    # 这里能不能用激活函数
    u_feature_layer = keras.layers.Dense(100, activation='tanh', name='u_feature_layer')(u_combine)
    print(u_feature_layer.shape)
    return u_feature_layer

In [29]:
b_dense = 4
def get_book_feature(b_isbn_embedd, b_author_embedd, b_year_embedd, b_publisher_embedd, b_title_embedd, b_blurb_embedd):
    # 首先对前4个特征连接Dense层
    b_isbn_dense = keras.layers.Dense(b_dense, activation='relu', kernel_regularizer=tf.nn.l2_loss, name='b_isbn_dense')(b_isbn_embedd)
    b_author_dense = keras.layers.Dense(b_dense, activation='relu', kernel_regularizer=tf.nn.l2_loss, name='b_author_dense')(b_author_embedd)
    b_year_dense = keras.layers.Dense(b_dense, activation='relu', kernel_regularizer=tf.nn.l2_loss, name='b_year_dense')(b_year_embedd)
    b_publisher_dense = keras.layers.Dense(b_dense, activation='relu', kernel_regularizer=tf.nn.l2_loss, name='b_publisher_dense')(b_publisher_embedd)
    # 合并这四个特征,  b_combine_four shape = (?, 1, 16)
    b_combine_four = keras.layers.concatenate([b_isbn_dense, b_author_dense, b_year_dense, b_publisher_dense], name='b_four_combine')
    print('b_combine_four.shape', b_combine_four.shape)
    # 对title进行卷积
    b_title_reshape = keras.layers.Lambda(lambda layer: tf.expand_dims(layer, 3))(b_title_embedd)  # shape=(?,10, 16, 1)
    print('b_title_reshape.shape = ', b_title_reshape.shape)
    # b_title_conv.shape = 
    b_title_conv = keras.layers.Conv2D(filters=8, kernel_size=(2, embed_dim_title), kernel_regularizer=tf.nn.l2_loss, strides=1)(b_title_reshape)# shape=(?, 14, 1, 8)
    # b-title_pool.shape =
    b_title_pool = keras.layers.MaxPool2D(pool_size=(9, 1), strides=1)(b_title_conv) # shape=(?,1, 1, 8)
    print('b_title_conv.shape = ', b_title_conv)
    print('b_title_pool.shape = ', b_title_pool)
    
    # 对blurb进行处理
    # shape = 
    b_blurb_lstm_1 = keras.layers.Bidirectional(tf.keras.layers.LSTM(16, name='b_blurb_lstm', dropout=0.5, return_sequences=True), merge_mode='concat')(b_blurb_embedd) 
    print('b_blurb_lstm_1.shape = ', b_blurb_lstm_1.shape)
    b_blurb_lstm_2 = keras.layers.LSTM(32, name='b_blurb_lstm', dropout=0.5, return_sequences=False)(b_blurb_lstm_1) 
    print('b_blurb_lstm_2.shape = ', b_blurb_lstm_2.shape)
    # 将title和blurb合并
    b_title_reshape = keras.layers.Reshape([b_title_pool.shape[3]])(b_title_pool)
    # b_combine_blurb_title.shape = 
    b_combine_blurb_title = keras.layers.concatenate([b_title_reshape, b_blurb_lstm_2], axis=1, name='b_combine_blurb_title')
    print('b_combine_blurb_title.shape = ', b_combine_blurb_title)
    b_blurb_title_dense = keras.layers.Dense(64, kernel_regularizer=tf.nn.l2_loss, activation='relu', name='b_blurb_title_dense')(b_combine_blurb_title)
#     b_blurb_title_dense_drop = keras.layers.Dropout(rate=0.5, name='b_blurb_title_dense_drop')(b_blurb_title_dense)
    b_blurb_title_dense = keras.layers.Dense(64, activation='relu', name='b_blurb_title_dense')(b_combine_blurb_title)
    # b_combine_four_reshape shape = (?, 64)
    b_combine_four_reshape = keras.layers.Reshape([b_combine_four.shape[2]], name='b_combine_four_reshape')(b_combine_four)
    # 合并所有的书籍特征
#     b_combine_book = keras.layers.concatenate([b_combine_blurb_title, b_combine_four_reshape], axis=1, name='b_combine_book')
    b_combine_book = keras.layers.concatenate([b_blurb_title_dense, b_combine_four_reshape], axis=1, name='b_combine_book')

    # 得到书籍矩阵
    b_feature_layer = keras.layers.Dense(100, name='b_feature_layer', activation='tanh')(b_combine_book)
    return b_feature_layer

In [30]:
def get_rating(user_feature, book_feature):
#     multiply_layer = keras.layers.Lambda(lambda layer: tf.reduce_sum(layer[0]+layer[1], axis=1, keepdims=True), name = 'user_book_feature')((user_feature, book_feature))
    inference_layer = keras.layers.concatenate([user_feature, book_feature], axis=1, name='user_book_feature')
    inference_dense = tf.keras.layers.Dense(64, kernel_regularizer=tf.nn.l2_loss, activation='relu')(inference_layer)
    multiply_layer = tf.keras.layers.Dense(1, name="inference")(inference_layer)  # inference_dense
    print(multiply_layer.shape)
    return multiply_layer

In [31]:
MODEL_DIR = './model/'

class model_network():
    def __init__(self):
        self.batchsize = 512
        self.epoch = 20
    def creat_model(self):
        user_id, user_location, book_isbn, book_author, book_year, book_publisher, book_title, book_blurb = get_inputs()
        user_id_embedd, user_loca_embedd = user_embed_layer(user_id, user_location)
        book_isbn_embedd, book_author_embedd, book_year_embedd, book_publisher_embedd, book_title_embedd, book_blurb_embedd = book_emded_layer(book_isbn, book_author, book_year, book_publisher, book_title, book_blurb)
        u_feature_layer = get_user_feature(user_id_embedd, user_loca_embedd)
        b_feature_layer = get_book_feature(book_isbn_embedd, book_author_embedd, book_year_embedd, book_publisher_embedd, book_title_embedd, book_blurb_embedd)
        multiply_layer = get_rating(u_feature_layer, b_feature_layer)
        model = keras.Model(inputs=[user_id, user_location, book_isbn, book_author, book_year, book_publisher, book_title, book_blurb],
                    outputs=[multiply_layer])
        return model
    def train_model(self):
        weights_path = './model_weights/model_2.hdf5'
#         checkpoint = keras.callbacks.ModelCheckpoint(filepath=weights_path, monitor='val_loss', mode='min', save_weights_only=True)
        model_optimizer = keras.optimizers.Adamax()
        model = self.creat_model()
        model.compile(optimizer=model_optimizer, loss=keras.losses.mse)
        history = model.fit(train_features, train_labels, validation_data=(val_features, val_lables), epochs=self.epoch, batch_size=self.batchsize, verbose=1)
        model.summary()
        return model, history
    def predict_model(self, model):
        test_loss = model.evaluate(test_features, test_lables, batch_size = 512, verbose=1)
        return test_loss

In [ ]:
net_work = model_network()
model, history = net_work.train_model()

(None, 64)
(None, 100)
b_combine_four.shape (None, 1, 16)
b_title_reshape.shape =  (None, 10, 16, 1)
b_title_conv.shape =  Tensor("conv2d/Identity:0", shape=(None, 9, 1, 8), dtype=float32)
b_title_pool.shape =  Tensor("max_pooling2d/Identity:0", shape=(None, 1, 1, 8), dtype=float32)
b_blurb_lstm_1.shape =  (None, 200, 32)
b_blurb_lstm_2.shape =  (None, 32)
b_combine_blurb_title.shape =  Tensor("b_combine_blurb_title/Identity:0", shape=(None, 40), dtype=float32)
(None, 1)
Train on 165218 samples, validate on 3442 samples
Epoch 1/20
165218/165218 [==============================] - 62s 377us/sample - loss: 26.3266 - val_loss: 17.0480
Epoch 2/20
165218/165218 [==============================] - 52s 313us/sample - loss: 15.1689 - val_loss: 13.2845
Epoch 3/20
165218/165218 [==============================] - 52s 312us/sample - loss: 13.2525 - val_loss: 12.4625
Epoch 4/20
165218/165218 [==============================] - 52s 315us/sample - loss: 12.6880 - val_loss: 12.1982
Epoch 5/20
165218/1652

In [ ]:
train_loss = history.history['loss']
val_loss = history.history['val_loss']
plt.figure(1)
plt.plot(train_loss, c='r', label='train_loss')
plt.plot(val_loss, c='b', label='val_loss')
plt.legend()
plt.xlim([1, 50])
plt.ylim([0, 20])
plt.show()

### 画出模型图

In [ ]:
keras.utils.plot_model(model, to_file='model_2.png', show_shapes=True, show_layer_names=True)